# 컨텐츠 기반 추천 시스템 구현

* id, 도수, 모양, 향, 나라, 맛, 대분류, 입넘김, 소분류 데이터 사용하면 될듯
* 지금 시간 지나서 좀 바뀌었는데 뭐 알아서 하겠지

In [ ]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg
import numpy as np

In [ ]:
# db에 있는거 바로 dataframe으로 바꿈
conn = psycopg.connect(host="sulleong.site", dbname="sulleong", user="sulleong", password="Sulleong104**", port=5432)
sql = "SELECT * FROM beer WHERE country IS NOT NULL;"
# sql = "select * from beer_merge_test;"
beers = sqlio.read_sql_query(sql, conn)
beers = beers.set_index('id')

## 전처리.....

In [ ]:
# drop table column
beers = beers.drop(columns=['description', 'name', 'name_kor'])

# nan값들을 0으로 바꿈, 단 abv는 않건드림
beers['appearance'].fillna(0, inplace=True)
beers['aroma'].fillna(0, inplace=True)
beers['flavor'].fillna(0, inplace=True)
beers['mouthfeel'].fillna(0, inplace=True)
beers['overall'].fillna(0, inplace=True)
beers['brewer_id'] = beers['brewer_id'].astype(str)


In [ ]:
# 저거 그냥 합이라 나눠줘야 함.... 나중에 스케일링도 또 해야할듯 ㅠㅠ
beers['appearance'] = beers.apply(lambda row: round(row['appearance'] / row['review_count'], 1) if row['review_count'] != 0 else row['appearance'], axis=1)
beers['aroma'] = beers.apply(lambda row: round(row['aroma'] / row['review_count'], 1) if row['review_count'] != 0 else row['aroma'], axis=1)
beers['flavor'] = beers.apply(lambda row: round(row['flavor'] / row['review_count'], 1) if row['review_count'] != 0 else row['flavor'], axis=1)
beers['mouthfeel'] = beers.apply(lambda row: round(row['mouthfeel'] / row['review_count'], 1) if row['review_count'] != 0 else row['mouthfeel'], axis=1)
beers['overall'] = beers.apply(lambda row: round(row['overall'] / row['review_count'], 1) if row['review_count'] != 0 else row['overall'], axis=1)

* 이제 문자 데이터는 OneHotEncoder 통해 행렬화 시켜서 보여줌
  * 범주형 데이터이기 때문에 CountVectorizer대신 이거 사용

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# encoder 설정, 바로 행렬에 붙일거라, 근데 데이터 바뀌어서 비어있는 곳이 있음;; 그래도 sparsematrix 안씀
encoder = OneHotEncoder(sparse_output=False)

# 숫자형 데이터
numeric_data = beers.drop(columns=['large_category', 'sub_category', 'country', 'brewer_id'])

# 범주형 데이터 맞추고 변환
encoder.fit(beers[['large_category', 'sub_category', 'country', 'brewer_id']])
encoding_beer = encoder.transform(beers[['large_category', 'sub_category', 'country', 'brewer_id']])
# # 다시 pandas 데이터프레임으로 변환해서 붙일수 있도록 만듬
encoded_categories_beer = pd.DataFrame(encoding_beer, columns=encoder.get_feature_names_out(input_features=['large_category', 'sub_category', 'country', 'brewer_id']), index=beers.index)
# print(processed_beer_notscaled)
processed_beer_notscaled = pd.concat([numeric_data, encoded_categories_beer], axis=1)

* 이제 모든 숫자들 0에서 1사이로 스케일링 함?
  * 일단 그냥 계산해보고 비교해봐야 할듯

In [ ]:
from sklearn.impute import SimpleImputer
# null값 처리해야 할듯;;; 그냥 평균값 넣음
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(processed_beer_notscaled['abv'].values.reshape(-1, 1))
processed_beer_notscaled['abv'] = imp.transform(processed_beer_notscaled['abv'].values.reshape(-1, 1))

In [ ]:
processed_beer_notscaled = processed_beer_notscaled.drop(columns=['class_name'])

## 이제 계산...

In [ ]:
from sklearn.neighbors import NearestNeighbors

# 일단 10개의 가장 가까운 이웃들을 찾음, 식들은 차차 변경 예정
nbrs = NearestNeighbors(n_neighbors=10).fit(processed_beer_notscaled)
distances, indices = nbrs.kneighbors(processed_beer_notscaled)


# df = pd.DataFrame(indices, index=beers.index)
# df_d = pd.DataFrame(distances)
 
# df.to_csv('./sample_recommend.csv')

In [ ]:
# indicies는 pandas 기본 index로 줘서 우리 index로 바꿔줄 필요가 있음
index_mapping = dict(zip(range(len(beers.index)), beers.index))
custom_indices = np.vectorize(index_mapping.get)(indices)

In [ ]:
df = pd.DataFrame(custom_indices, index=beers.index)
df_d = pd.DataFrame(distances, index=beers.index)

In [ ]:
result = []
for i in df.index:
    for j in range(1, len(df.columns)):
        result.append([i, df.loc[i, j], df_d.loc[i, j]])

result_df = pd.DataFrame(result, columns=["beer", "rec_beer", "distance"])
result_df.to_csv('./cbf_result_1539.csv')